In [4]:
!pip install -U sentence-transformers
!pip install rank_bm25


[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# **Library Setup**

In [23]:
import pandas as pd
import numpy as np
import json
import re
import os
import warnings
import random
from datetime import datetime

# TEXT PROCESSING & SCRAPING 
import requests
from bs4 import BeautifulSoup

# MACHINE LEARNING & DENSE RETRIEVAL
import torch
from sentence_transformers import SentenceTransformer, util

# SPARSE RETRIEVAL
from rank_bm25 import BM25Okapi

# CONFIGURATION 
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 50) 
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("All Libraries (Standard + Retrieval) Imported Successfully!")

All Libraries (Standard + Retrieval) Imported Successfully!


# **Data Load and Preprocessing**

In [24]:
# Data_Load
try:
    df = pd.read_json('Realtime_Jobs_Data.json')
    
    print(f"Total Rows: {df.shape[0]}, Total Columns: {df.shape[1]}")

    # সব কলামের নাম প্রিন্ট করা
    print("\nExisting Columns:")
    print(df.columns.tolist())

    # প্রথম ৩টি রো দেখা
    print("\nData Preview:")
    display(df.head(3)) # জুপিটারে print এর চেয়ে display() ভালো দেখায়

except ValueError:
    print("❌ Error: 'Realtime_Jobs_Data.json' ফাইলটি পাওয়া যাচ্ছে না।")

# Mapping all columns
rename_map = {
    'JobId': 'job_id', 'CompnayName': 'company_name', 'CompanyID': 'company_id', 'CategoryID': 'category_id',
    'JobTitle': 'job_title', 'JobDescription': 'job_description', 'JobNature': 'job_type',
    'JobWorkPlace': 'work_place', 'PostedOn': 'posted_on', 'Deadline': 'deadline',
    'JobVacancies': 'vacancies', 'JobLocation': 'raw_location', 'EducationRequirements': 'raw_education',
    'SkillsRequired': 'skills', 'SuggestedSkills': 'suggested_skills', 'experience': 'experience',
    'Age': 'age', 'Gender': 'gender', 'JobSalaryMinSalary': 'min_salary', 'JobSalaryMaxSalary': 'max_salary',
    'JobSalaryRange': 'salary_range', 'ApplyURL': 'apply_url', 'ApplyEmail': 'apply_email',
    'ApplyInstruction': 'apply_instruction', 'CompanyWeb': 'company_web', 'CompanyAddress': 'company_address',
    'CompanyBusiness': 'company_business', 'JobOtherBenifits': 'job_benefits', 'JobSource': 'job_source',
    'OnlineApply': 'online_apply'
}

# নাম পরিবর্তন করা (কিন্তু সব কলাম রাখা)
df = df.rename(columns=rename_map)

# ID গুলোকে স্ট্রিং বানানো (সায়েন্টিফিক নোটেশন এড়ানোর জন্য)
for col in ['job_id', 'company_id', 'category_id']:
    if col in df.columns:
        df[col] = df[col].astype(str)

print("Column names standardized!")

#Cleaning HTML contents
def clean_html_text(text):
    if not isinstance(text, str): return text
    if pd.isna(text) or text == "": return None
    try:
        soup = BeautifulSoup(text, "html.parser")
        text = soup.get_text(separator=" ")
    except: pass
    return re.sub(r'\s+', ' ', text).strip()

# যেসব কলামে HTML ট্যাগ থাকতে পারে
text_cols = ['job_description', 'job_benefits', 'apply_instruction', 'raw_education', 'skills', 'suggested_skills', 'company_business']

print("Cleaning HTML tags...")

for col in text_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_html_text)

# গারবেজ ভ্যালু নাল (NaN) করে দেওয়া
missing_indicators = ["--", "N/A", "n/a", "Not Applicable", "[]", "", " ", "0000-00-00", "Any"]
df.replace(missing_indicators, np.nan, inplace=True)

print("HTML Cleaning Done!")

# Feature Extraction (Age, Loc, Edu)
# Helper Functions
def extract_range(text):
    """Age বা Experience থেকে min/max বের করে"""
    if pd.isna(text): return None, None
    numbers = re.findall(r'\d+', str(text))
    if not numbers: return None, None
    nums = [int(n) for n in numbers]
    if len(nums) >= 2: return nums[0], nums[1]
    elif len(nums) == 1: return nums[0], None
    return None, None

def extract_loc_edu(row):
    """Location & Education Logic"""
    raw_loc = str(row.get('raw_location', '')).strip()
    raw_edu = str(row.get('raw_education', '')).lower().strip()
    
    # Location
    dist, fine = raw_loc, "none"
    if "anywhere in bangladesh" in raw_loc.lower():
        dist = "Anywhere in Bangladesh"
    elif "(" in raw_loc:
        parts = re.findall(r"([^(]+)\s*\(([^)]+)\)", raw_loc)
        if parts: dist, fine = parts[0][0].strip(), parts[0][1].strip()

    # Education
    level, subject = "Any", "Any"
    if any(x in raw_edu for x in ['cse', 'computer science', 'it', 'software']):
        level, subject = "Bachelor", "Computer Science"
    elif any(x in raw_edu for x in ['bba', 'mba', 'business']):
        level, subject = "Bachelor", "Business Administration"
    elif 'diploma' in raw_edu:
        level, subject = "Diploma", "Engineering"
    
    return pd.Series([dist, fine, level, subject])

# --- Applying Logic ---
print("Generating new columns...")

# 1. Location & Education
df[['district', 'fine_grained_location', 'education_level', 'education_subject']] = df.apply(extract_loc_edu, axis=1)

# 2. Age
if 'age' in df.columns:
    age_data = df['age'].apply(extract_range)
    df['min_age'] = age_data.apply(lambda x: x[0])
    df['max_age'] = age_data.apply(lambda x: x[1])

# 3. Experience
if 'experience' in df.columns:
    exp_data = df['experience'].apply(extract_range)
    df['min_experience'] = exp_data.apply(lambda x: x[0])
    df['max_experience'] = exp_data.apply(lambda x: x[1])

# 4. Salary & Vacancies Fix
if 'vacancies' in df.columns:
    df['vacancies'] = pd.to_numeric(df['vacancies'], errors='coerce')
for col in ['min_salary', 'max_salary']:
    if col in df.columns: df[col] = df[col].replace(0, np.nan)

print("New Features Created: min_age, district, education_level, etc.")

import pandas as pd

# ==========================================
# 1. FIXED CATEGORY MAPPING (Based on Analysis)
# ==========================================
# আমরা ডেটা এনালাইসিস করে এই ম্যাপটি তৈরি করেছি
fixed_category_map = {
    # --- Functional Categories ---
    '1': 'Accounting/Finance',
    '2': 'Bank/Non-Bank Fin. Inst.',
    '3': 'Supply Chain/Procurement',
    '4': 'Education/Training',
    '5': 'Engineer/Architects',
    '6': 'Garments/Textile',
    '7': 'Gen Mgt/Admin',
    '8': 'IT & Telecommunication',
    '9': 'Marketing/Sales',
    '10': 'Digital Marketing/SEO',
    '11': 'Medical/Pharma',
    '12': 'NGO/Development',
    '13': 'Research/Consultancy',
    '14': 'Receptionist/Front Desk',
    '15': 'Data Entry/Operator',
    '16': 'Customer Support/Call Center',
    '17': 'HR/Org. Development',
    '18': 'Design/Creative',
    '19': 'Production/Operation',
    '20': 'Immigration/Visa Consultant',
    '22': 'Law/Legal',
    '24': 'Security/Support Service',
    '26': 'Agro (Plant/Animal/Fisheries)',
    '27': 'Commercial/Logistics',
    '28': 'Secretariat/Media',
    '29': 'Pharma/Medical Promo',
    
    # --- Blue Collar / Skilled Trade ---
    '61': 'Computer Operator',
    '62': 'Electrician/Technician',
    '63': 'Nurse/Patient Care',
    '64': 'Hotel/Restaurant/Chef',
    '65': 'Lab/Radiographer',
    '66': 'Electronics/Technician',
    '67': 'Driver/Motor Technician',
    '68': 'Chef/Cook',
    '69': 'Housekeeping/Domestic',
    '70': 'Security Guard',
    '71': 'Graphic Design/Video Editor',
    '72': 'Welder/Technical',
    '74': 'Garments Operator',
    '75': 'Labour/Helper',
    '76': 'CAD/Draftsman',
    '77': 'Delivery Man',
    '78': 'Machine Operator',
    '79': 'Peon/Messenger',
    '80': 'Cleaner/Support Staff',
    '81': 'Gardener/Mali',
    '82': 'Carpenter',
    '83': 'Salesman/Salesgirl',
    '84': 'Sales Representative (SR)',
    '85': 'Religious/Imam/Moazzin',
    '86': 'Sports/Fitness Trainer',
    '87': 'Interpreter/Translator',
    '88': 'Beautician',
    '89': 'Fire/Safety',
    '90': 'Boiler Operator',
    '91': 'Caregiver/Nanny',
    '92': 'Physiotherapist/Therapist',
    
    # --- Catch-All ---
    '0': 'General/Others',
    '-10': 'Logistics/Transport'
}

print("✅ Fixed Category Map Loaded.")

# ==========================================
# 2. APPLY MAPPING TO DATAFRAME
# ==========================================
# ম্যাপ অনুযায়ী নাম বসানো (যদি কোনো আইডি ম্যাপে না থাকে, 'Other' বসবে)
df['category_name'] = df['category_id'].astype(str).map(fixed_category_map).fillna('Other Category')

print("✅ Category Names Applied Successfully!")

# ==========================================
# 3. VERIFY RESULT
# ==========================================
# স্যাম্পল চেক করা
cols_to_view = ['category_id', 'category_name', 'job_title']
print("\nSample Data with Fixed Categories:")
print(df[cols_to_view].drop_duplicates('category_id').head(10))

# ==========================================
# CLEANING GARBAGE DATA (Bangla Jobs)
# ==========================================
print(f"Original Row Count: {len(df)}")

# ১. 'Bangla Job' লেখা আছে এমন সব রো খুঁজে বের করা
garbage_indices = df[df['job_title'].str.contains("Bangla Job", case=False, na=False)].index

# ২. এগুলো ড্রপ করে দেওয়া
df_clean = df.drop(garbage_indices)

# মেইন ডেটাফ্রেমে আপডেট করা
df = df_clean.copy()

print(f"Removed {len(garbage_indices)} 'Bangla Job' rows.")
print(f"Clean Row Count: {len(df)}")

# চেক করে দেখা যে আর আছে কিনা
remaining_garbage = df[df['job_title'].str.contains("Bangla Job", case=False, na=False)]
if remaining_garbage.empty:
    print("✅ All 'Bangla Job' garbage successfully removed!")
else:
    print("⚠️ Still some garbage left!")

import pandas as pd

# Final Features Selection
# ==========================================
# 0. DISPLAY SETTINGS (Safety First)
# ==========================================
# টেবিল যাতে ভেঙে না যায়, তাই আগে ডিসপ্লে সেট করে নিচ্ছি
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# ==========================================
# 1. CUSTOM ID GENERATION
# ==========================================
print("🛠️ Generating Custom IDs...")

# Job ID
df['job_id'] = [f"JOB-{i+1:05d}" for i in range(len(df))]

# Company ID
unique_companies = df['company_name'].unique()
company_id_map = {name: f"COM-{5000+i}" for i, name in enumerate(unique_companies)}
df['company_id'] = df['company_name'].map(company_id_map)

print("✅ Custom Job & Company IDs Created.")


# ==========================================
# 2. FINAL COLUMN SELECTION
# ==========================================
# এই লিস্টে category_name যোগ করা হয়েছে ঠিক category_id এর পরে
target_columns = [
    # IDs & Category
    'job_id', 'company_id', 'category_id', 'category_name', 
    
    # Company Info
    'company_name', 'company_web', 'company_address', 'company_business',
    
    # Core Job Info
    'job_title', 'job_description', 'job_type', 'work_place',
    'job_benefits', 'job_source',
    
    # Dates & Vacancy
    'posted_on', 'deadline', 'vacancies',
    
    # Skills & Gender
    'skills', 'suggested_skills', 'gender',
    
    # Application Details
    'apply_url', 'apply_email', 'apply_instruction', 'online_apply',
    
    # Salary
    'min_salary', 'max_salary',
    
    # Processed Location
    'district', 'fine_grained_location',
    
    # Processed Education
    'education_level', 'education_subject',
    
    # Processed Age & Experience
    'min_age', 'max_age', 
    'min_experience', 'max_experience'
]

# সেইফলি কলাম সিলেক্ট করা (যদি category_name আগের স্টেপে তৈরি না হয়ে থাকে, তবে এরর দিবে না)
existing_cols = [c for c in target_columns if c in df.columns]
df_final = df[existing_cols].copy()

# ==========================================
# 3. FINAL OUTPUT PREVIEW
# ==========================================
print("\n" + "="*40)
print("🎉 FINAL DATASET READY")
print("="*40)
print(f"Total Columns: {len(df_final.columns)}")
print(f"Total Rows: {len(df_final)}")

# আমরা এখন স্পেসিফিক কলামগুলো দেখবো যাতে নিশ্চিত হওয়া যায় অর্ডার ঠিক আছে
display_cols = ['job_id', 'category_id', 'category_name', 'job_title', 'company_name']
print("\nSample Data (Checking Category Order):")
# জুপিটারে display() ফাংশন টেবিল সুন্দর দেখায়
try:
    display(df_final[display_cols].head(5))
except:
    print(df_final[display_cols].head(5))

df_final.head(10)

Total Rows: 13597, Total Columns: 70

Existing Columns:
['test', 'JobId', 'JobFound', 'error', 'CompnayName', 'JobTitle', 'PostedOn', 'Deadline', 'DeadlineDB', 'JobVacancies', 'JobDescription', 'JobNature', 'JobWorkPlace', 'EducationRequirements', 'SkillsRequired', 'SuggestedSkills', 'Publications', 'Age', 'experience', 'Gender', 'AdditionJobRequirements', 'JobLocation', 'OnlineApply', 'CompanyBusiness', 'CompanyAddress', 'CompanyHideAddress', 'CompanyWeb', 'JobAppliedEmail', 'JobSource', 'JobOtherBenifits', 'RecruitmentProcessingInformation', 'RecruitingCompanysProfile', 'JobSalaryRange', 'JobSalaryRangeText', 'JobSalaryMinSalary', 'JobSalaryMaxSalary', 'ShowSalary', 'overseasnote', 'JobAdType', 'JobLOgoName', 'JobKeyPoints', 'ApplyInstruction', 'ApplyEmail', 'HardCopy', 'WalkInInterview', 'ApplyURL', 'Photograph', 'PhotographMsg', 'JObIMage', 'upcoming', 'upcomingln', 'CompanyOtherJ0bs', 'CompanyID', 'CompanyNameENG', 'AssessmentRequired', 'Context', 'RLNO', 'PreferVideoResume', 'Att

,test,JobId,JobFound,error,CompnayName,JobTitle,PostedOn,Deadline,DeadlineDB,JobVacancies,JobDescription,JobNature,JobWorkPlace,EducationRequirements,SkillsRequired,SuggestedSkills,Publications,Age,experience,Gender,AdditionJobRequirements,JobLocation,OnlineApply,CompanyBusiness,CompanyAddress,CompanyHideAddress,CompanyWeb,JobAppliedEmail,JobSource,JobOtherBenifits,RecruitmentProcessingInformation,RecruitingCompanysProfile,JobSalaryRange,JobSalaryRangeText,JobSalaryMinSalary,JobSalaryMaxSalary,ShowSalary,overseasnote,JobAdType,JobLOgoName,JobKeyPoints,ApplyInstruction,ApplyEmail,HardCopy,WalkInInterview,ApplyURL,Photograph,PhotographMsg,JObIMage,upcoming,upcomingln,CompanyOtherJ0bs,CompanyID,CompanyNameENG,AssessmentRequired,Context,RLNO,PreferVideoResume,AttachedResume,bottomAlertMsg,ProUser,ApplicantMatchingScore,CategoryID,C2C,MobileNo,CONFIDENTIAL,NewspaperJob,Closed,ApplyRedirectUrl,AccessibilityAware
0,1,1436370,True,0,RK Supply Ltd.,Data Analyst / Accounts Officer,"Dec 6, 2025","Dec 31, 2025",12/31/2025 00:00:00,6,"<p><strong><span style=""color:rgb(51, 51, 51);...",Full Time,Work at office,<ul><ul><li>Bachelor of Business Administratio...,"Accounting,Accounting Data Entry,Canva Pro,Dat...","Accounting Data Entry,Accounting Software,Acco...",0,20 to 35 years,<ul><li>At least 1 year</li><li>The applicants...,"M,F",<ul><li>Age 20 to 35 years</li></ul>,Dhaka (Banani),True,,,True,,,,,,,Tk. 20000 - 35000 (Monthly),,20000,35000,1,,1,,"<p><strong><span style=""color:rgb(51, 51, 51)""...",Selected candidates for this role will be trai...,,,,,-,-,,,,0,109093,RK Supply Ltd.,No,,,0,0,বিডিজবস-এ প্রকাশিত যেকোনো চাকরি সংক্রান্ত তথ্য...,0,0,8,0,,,False,0,//mybdjobs.bdjobs.com/mybdjobs/signin.asp?c7`6...,[]
1,1,1437042,True,0,Pridesys IT Limited,Processor/Senior Processor,"Dec 6, 2025","Jan 5, 2026",01/05/2026 00:00:00,2,<ul><li>Review property preservation work orde...,Full Time,Work at office,<ul><ul><li>Bachelor/Honors</li></ul></ul>,"Flexibility and Adaptability,Google Sheets,MS ...","Client Service,Property Management,Property Pr...",0,20 to 35 years,<ul><li>At most 2 years</li><li>Freshers are a...,M,<ul><li>Age 20 to 35 years</li><li>Only Male</...,Dhaka (Kawran Bazar),True,Pridesys IT Ltd. own developed ERP product for...,"Level-11, Vision 2021 Tower-1, Software Techno...",False,,,,<ul><li>Salary Review: Yearly</li><li>Festival...,,,Negotiable,,0,0,1,,1,https://corporate.bdjobs.com/logos/38072_0.png,,,,,,,-,-,,,,0,38072,Pridesys IT Limited,No,,,0,0,বিডিজবস-এ প্রকাশিত যেকোনো চাকরি সংক্রান্ত তথ্য...,0,0,8,0,,,False,0,//mybdjobs.bdjobs.com/mybdjobs/signin.asp?a4]7...,[]
2,1,1437039,True,0,Aalok Healthcare & Hospital,Assistant Manager (Customer Care / Admin),"Dec 6, 2025","Dec 26, 2025",12/26/2025 00:00:00,4,<h3>Shift Management and Operational Leadershi...,Full Time,Work at office,<ul><ul><li>Masters</li></ul></ul>,"Computer Literacy,Customer Service,Health/ Med...","Administration,Complaint Management,Hospital B...",0,Na,<ul><li>3 to 5 years</li><li>The applicants sh...,M,<ul><li>Only Male</li></ul> <ul><li><p>3 to 5 ...,Dhaka (Mirpur),True,,"Corporate Office: House# 3, Road # 4, Block # ...",False,https://www.aalokhealthcare.com/,,,<ul><li>Provident fund</li><li>Salary Review: ...,,,--,,0,0,0,,1,https://corporate.bdjobs.com/logos/92299_3.png,,<p>Read before Apply and email us mention as E...,Send your CV to the given email hr.aalok@gmail...,,,,-,-,,,,4,92299,Aalok Healthcare & Hospital,No,,,1,0,বিডিজবস-এ প্রকাশিত যেকোনো চাকরি সংক্রান্ত তথ্য...,0,0,11,0,,,False,0,//mybdjobs.bdjobs.com/mybdjobs/signin.asp?3_7a...,[{'Name': 'Do you have Disability Inclusion Po...


Column names standardized!
Cleaning HTML tags...
HTML Cleaning Done!
Generating new columns...
New Features Created: min_age, district, education_level, etc.
✅ Fixed Category Map Loaded.
✅ Category Names Applied Successfully!

Sample Data with Fixed Categories:
   category_id                  category_name                                          job_title
0            8         IT & Telecommunication                    Data Analyst / Accounts Officer
2           11                 Medical/Pharma          Assistant Manager (Customer Care / Admin)
3           84      Sales Representative (SR)                          Sales Representative (SR)
4            9                Marketing/Sales  Sales Hero (Sales Executive - Freshers) RANGS ...
5           17            HR/Org. Development                                HR Manager - Female
6            6               Garments/Textile  Senior Executive - Dyeing (Textile Division, G...
8           19           Production/Operation             O

,job_id,category_id,category_name,job_title,company_name
0,JOB-00001,8,IT & Telecommunication,Data Analyst / Accounts Officer,RK Supply Ltd.
1,JOB-00002,8,IT & Telecommunication,Processor/Senior Processor,Pridesys IT Limited
2,JOB-00003,11,Medical/Pharma,Assistant Manager (Customer Care / Admin),Aalok Healthcare & Hospital
3,JOB-00004,84,Sales Representative (SR),Sales Representative (SR),Eureka food and beverage
4,JOB-00005,9,Marketing/Sales,Sales Hero (Sales Executive - Freshers) RANGS ...,Rancon Holdings Limited


,job_id,company_id,category_id,category_name,company_name,company_web,company_address,company_business,job_title,job_description,job_type,work_place,job_benefits,job_source,posted_on,deadline,vacancies,skills,suggested_skills,gender,apply_url,apply_email,apply_instruction,online_apply,min_salary,max_salary,district,fine_grained_location,education_level,education_subject,min_age,max_age,min_experience,max_experience
0,JOB-00001,COM-5000,8,IT & Telecommunication,RK Supply Ltd.,NaN,NaN,None,Data Analyst / Accounts Officer,"RK Supply Ltd. (Company No. 09740433), establi...",Full Time,Work at office,None,NaN,"Dec 6, 2025","Dec 31, 2025",6.0,"Accounting,Accounting Data Entry,Canva Pro,Dat...","Accounting Data Entry,Accounting Software,Acco...","M,F",NaN,NaN,Selected candidates for this role will be trai...,True,20000.0,35000.0,Dhaka,Banani,Bachelor,Computer Science,20.0,35.0,1.0,2.0
1,JOB-00002,COM-5001,8,IT & Telecommunication,Pridesys IT Limited,NaN,"Level-11, Vision 2021 Tower-1, Software Techno...",Pridesys IT Ltd. own developed ERP product for...,Processor/Senior Processor,Review property preservation work orders submi...,Full Time,Work at office,Salary Review: Yearly Festival Bonus: 2 Dinner...,NaN,"Dec 6, 2025","Jan 5, 2026",2.0,"Flexibility and Adaptability,Google Sheets,MS ...","Client Service,Property Management,Property Pr...",M,NaN,NaN,None,True,NaN,NaN,Dhaka,Kawran Bazar,Any,Any,20.0,35.0,2.0,NaN
2,JOB-00003,COM-5002,11,Medical/Pharma,Aalok Healthcare & Hospital,https://www.aalokhealthcare.com/,"Corporate Office: House# 3, Road # 4, Block # ...",None,Assistant Manager (Customer Care / Admin),Shift Management and Operational Leadership (2...,Full Time,Work at office,Provident fund Salary Review: Yearly Festival ...,NaN,"Dec 6, 2025","Dec 26, 2025",4.0,"Computer Literacy,Customer Service,Health/ Med...","Administration,Complaint Management,Hospital B...",M,NaN,Send your CV to the given email hr.aalok@gmail...,Read before Apply and email us mention as Expe...,True,NaN,NaN,Dhaka,Mirpur,Any,Any,NaN,NaN,3.0,5.0
3,JOB-00004,COM-5003,84,Sales Representative (SR),Eureka food and beverage,NaN,"Jong & sons market,Anwar jong road,Ashulia, Sa...","We manufacture, market, and sell all types of ...",Sales Representative (SR),দায়িত্ব ও কর্তব্য: কোম্পানির পণ্য বিক্রয় করা...,Full Time,NaN,অন্যান্য সুবিধা: আলোচনা সাপেক্ষে,NaN,"Dec 6, 2025","Jan 5, 2026",NaN,None,"Market Research,Marketing,Sales & Marketing,Sa...",M,NaN,NaN,আগ্রহী প্রার্থীদের আবেদন করার জন্য উৎসাহিত করা...,True,18000.0,22000.0,Anywhere in Bangladesh,none,Any,Any,22.0,40.0,2.0,5.0
4,JOB-00005,COM-5004,9,Marketing/Sales,Rancon Holdings Limited,NaN,"117/A Old Airport Road, Bijoy Sharani","A local conglomerate, doing diversified busine...",Sales Hero (Sales Executive - Freshers) RANGS ...,Greet and assist walk-in customers with a frie...,Full Time,Work at office,"T/A,Mobile bill,Performance bonus Salary Revie...",NaN,"Dec 6, 2025","Dec 31, 2025",20.0,None,"Marketing and Sales,Retail Sales,Sales,Sales &...",M,NaN,NaN,None,True,20000.0,20000.0,Anywhere in Bangladesh,none,Any,Any,24.0,30.0,NaN,NaN
5,JOB-00006,COM-5005,17,HR/Org. Development,ALTERNATIVE RECRUITMENT LTD.,NaN,"BNS Center, Level 08\r\nSector 07\r\nUttara ,D...",Alternative Recruitment is a certified Educati...,HR Manager - Female,Join Our Global Team – Female HR Manager Wante...,Full Time,"Work from home,Work at office","Profit share,Mobile bill Salary Review: Yearly...",NaN,"Dec 6, 2025","Jan 5, 2026",15.0,"Computer Literacy,English typing,FLUENCY IN EN...","Human Resource Management,Management,Recruitme...",F,NaN,Send your CV to the given email hiring@thealte...,Please read carefully : This position is open ...,True,20000.0,30000.0,Dhaka,"Uttara Sector 10, Uttara Sector 12",Diploma,Engineering,25.0,38.0,1.0,5.0
6,JOB-00007,COM-5006,6,Garments/Textile,Epyllion Group,NaN,"Corporate Office: NINAKABBO, Level: 12, 227/A,...","Textile, Garments, Garments Accessories, Washi...","Senior Executive - Dyeing (Textile Divisi

# **Dense Retreival**

# **1st Step**

In [25]:
def create_improved_context(row):
    # অভিজ্ঞতার তথ্যকে টেক্সট হিসেবে সাজানো (যেমন: 2 years experience)
    experience_text = f"{row['min_experience']} to {row['max_experience']} years experience" if row['min_experience'] != "" else "Freshers"
    
    parts = [
        str(row['job_title']),
        str(row['category_name']), # নতুন
        str(row['company_name']),  # নতুন
        BeautifulSoup(str(row['job_description']), 'html.parser').get_text(),
        str(row['skills']),
        str(row['job_type']),       # নতুন
        str(row['education_level']),
        str(row['education_subject']),
        str(row['district']),
        str(row['fine_grained_location']), # নতুন
        experience_text             # নতুন
    ]
    
    # সব টেক্সট ছোট হাতের অক্ষরে এবং অতিরিক্ত স্পেস রিমুভ করে রিটার্ন করা
    combined = " ".join([str(p) for p in parts if p]).lower()
    return combined

# নতুন কলাম তৈরি
df_final['combined_text'] = df_final.apply(create_improved_context, axis=1)

# আগের মতোই পাঙ্কচুয়েশন রিমুভ করা (মডেলকে ক্লিন ডাটা দেওয়ার জন্য)
df_final['combined_text'] = df_final['combined_text'].str.replace(r'[^\w\s]', ' ', regex=True)

# Verify the result
print("Combined Text Sample:")
print(df_final['combined_text'].iloc[0][:200] + "...")

Combined Text Sample:
data analyst   accounts officer it   telecommunication rk supply ltd  rk supply ltd   company no  09740433   established in august 2015 and based in london  is a leading uk supplier of sim cards and m...


# ** step -02**

In [26]:
from sentence_transformers import SentenceTransformer
import torch

# 1. Load the model (this stays the same)
model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Encode the job data - MAKE SURE TO USE df_final
# This might take a few minutes if you have 10,000+ jobs
job_embeddings = model.encode(
    df_final['combined_text'].tolist(), 
    show_progress_bar=True, 
    convert_to_tensor=True
)

print("✅ Job Embeddings Generated Successfully!")

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 276.13it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Batches: 100%|██████████| 423/423 [11:35<00:00,  1.64s/it]

✅ Job Embeddings Generated Successfully!


# **Step 1: Tokenization**

In [27]:
from rank_bm25 import BM25Okapi

# Simple tokenization: lowercase and split by space
tokenized_corpus = [doc.lower().split(" ") for doc in df_final['combined_text'].tolist()]
bm25 = BM25Okapi(tokenized_corpus)

# **Step 3: Creating the Retrieval Logic (The "Super Function")**

In [28]:
from sentence_transformers import util

def get_recommendations(query, k=5):
    # --- 1. Dense Search ---
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, job_embeddings)[0]
    top_dense_indices = torch.topk(cos_scores, k=k).indices.tolist()
    
    # --- 2. Sparse Search ---
    tokenized_query = query.lower().split(" ")
    top_sparse_results = bm25.get_top_n(tokenized_query, df.to_dict('records'), n=k)
    
    return {
        "dense_results": df.iloc[top_dense_indices][['job_title', 'company_name']],
        "sparse_results": top_sparse_results
    }

# Example Usage
# results = get_recommendations("Looking for a data analyst job in Dhaka")

# **হাইব্রিড সার্চ ফাংশন (The Solution)**

In [38]:
def get_hybrid_recommendations(query, k=5, alpha=0.5):
    """
    alpha: Dense এবং Sparse এর গুরুত্ব ঠিক করে। 
    0.5 মানে দুইটাই সমান গুরুত্বপূর্ণ।
    """
    # 1. Dense Score (Cosine Similarity)
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, job_embeddings)[0].cpu().numpy()
    
    # 2. Sparse Score (BM25)
    tokenized_query = query.lower().split()
    bm25_scores = bm25.get_scores(tokenized_query)
    
    # ৩. Normalization (স্কোরগুলোকে ০ থেকে ১ এর মধ্যে আনা)
    # Dense scores are already roughly 0-1, but BM25 isn't.
    if np.max(bm25_scores) != 0:
        bm25_scores = bm25_scores / np.max(bm25_scores)
    
    # ৪. Hybrid Score calculation
    # Formula: $Score = \alpha \times \text{Dense} + (1 - \alpha) \times \text{Sparse}$
    final_scores = (alpha * cos_scores) + ((1 - alpha) * bm25_scores)
    
    # ৫. Top K indices বের করা
    top_indices = np.argsort(final_scores)[::-1][:k]
    
    return df_final.iloc[top_indices][['job_title', 'company_name', 'district', 'skills', 'job_type','fine_grained_location']]

# টেস্ট করার নিয়ম
#results = get_hybrid_recommendations("Looking for a Python Developer job in Dhaka", k=5)
# display(results)

In [44]:
results = get_hybrid_recommendations("Part time Software Engineer", k=5)
display(results)

,job_title,company_name,district,skills,job_type,fine_grained_location
3227,Software Engineer (.NET Core & Angular),HawarIT Software Service Ltd.,Dhaka,"Angular,ASP.NET Core,ASP.NET Core MVC,ASP.NET ...",Full Time,none
2865,Software Engineer (Full-Stack),mPower Social Enterprises Ltd.,Dhaka,None,Full Time,Gulshan
6922,Architect / Civil Engineer,A Reputed Import Machineries Company,Dhaka,"Architecture, Civil Engineering",Full Time,Mirpur
12357,Asst. Software Engineer/Software Engineer,A Reputed Garments Manufacturing Company in Ba...,Dhaka,"ASP.NET Core, JavaScript, React | Node JS | Ja...",Full Time,none
13206,Software Engineer ( .Net ),Leading Multinational Software Company,Anywhere in Bangladesh,Dot Net,Full Time,none
